# NeRF（Neural Radiance Field）

- Reference：[【较真系列】讲人话-NeRF全解（原理+代码+公式）](https://www.bilibili.com/video/BV1CC411V7oq/)

- 广义理解定义
  - 使用神经网络（MLP）来 *隐式地* 存储 3D 信息
    - 显式的 3D 信息：有明确的 x，y，z 的值（mesh，voxel，点云…）
    - 隐式的 3D 信息：无明确的 x，y，z 的值，只能输出指定角度的 2D 图片
  - 训练时，使用给定静止场景下的若干张图片
  - 推论：
    - 模型 **不具有** 泛化能力
    - 一个模型只能存储一个 3D 信息
  - NeRF 是一个 8 层的 MLP

- 相机模型：连接 3D 世界与 2D 图片
  - 坐标系
    - ![坐标系](https://learnopengl.com/img/getting-started/coordinate_systems.png)
    - 模型坐标系（Model-space Coordinate，Local Coordinate）
    - 世界坐标系（World-space Coordinate）
    - 相机坐标系（Camera-space Coordinate，View-space Coordinate）
    - 归一化相机坐标系（Normalized Device Coordinate，NDC，Clip-space Coordinate）
    - 像素坐标系（Screen-space Coordinate）
    - $\mathbf{V}_{\mathrm{clip}} \in [-1, 1]^3 = \mathbf{P} \, \mathbf{V} \,  \mathbf{M} \, \left( \mathbf{V}_{\mathrm{local}} \in \mathbb{R}^3 \right)$
  - 视锥和坐标变换
    - ![视锥](./view_frustum.jpg)
    - Perspective projection is to *stretch* the view frustum into a canonical view volume. 
    - The common practive is to map view-space coordinate $(x, y, z, 1)^\top$ into NDC $(x^{'}, y^{'}, z^{'}, z)^\top = (x_p, y_p, z_p, 1)^\top$.
      - For $x$ and $y$: 
        - The near plane is bounded by $(l, r, b, t)$ where $z = n$;
        - An internal slice inside the frustum where $z = z$ should be bounded by $\left( \dfrac{z}{n} l, \dfrac{z}{n} r, \dfrac{z}{n} b, \dfrac{z}{n} t \right)$;
        - This internal slice should be stretched into $[-1, 1, -1, 1]$;
        - This yields the first two lines in the following equation.
          - Note that the coefficients for $z$ are handled by the $w$ dimension in the actual projection matrix. 
      - For $z$:
        - We need to map $z$ from $[n, f]$ into $[-1, 1]$.
        - This yields the last three lines in the following equation. 
    - 那么：
    $
    \left\{
    \begin{aligned}
    \dfrac{x - \dfrac{z}{n} l }{r - l} & = \dfrac{x_p + 1}{2} \\
    \dfrac{y - \dfrac{z}{n} b }{t - b} & = \dfrac{y_p + 1}{2} \\
    z_p & = \dfrac{A z + B}{z} \\
    z_p(n) & = -1 \\
    z_p(f) & = 1
    \end{aligned}
    \right.
    ;
    $
    - 解得投影变换矩阵如下：
    $
    \mathbf{P} = 
    \begin{pmatrix}
       \dfrac{2n}{r - l} & 0 & -\dfrac{r + l}{r - l} & 0 \\
       0 & \dfrac{2n}{t - b} & -\dfrac{t + b}{t - b} & 0 \\ 
       0 & 0 & -\dfrac{n + f}{n - f} & \dfrac{2nf}{n - f} \\ 
       0 & 0 & 1 & 0
    \end{pmatrix}
    $
    - 特别地，如果视锥是对称的（$-l = r = \dfrac{w}{2}, -b = t = \dfrac{h}{2}$），则
    $
    \mathbf{P} = 
    \begin{pmatrix}
       \dfrac{2n}{w} & 0 & 0 & 0 \\
       0 & \dfrac{2n}{h} & 0 & 0 \\ 
       0 & 0 & -\dfrac{n + f}{n - f} & \dfrac{2nf}{n - f} \\ 
       0 & 0 & 1 & 0
    \end{pmatrix}
    $  

- 体渲染
  - 属于渲染技术的分支
  - 目的是解决云、烟、果冻等 *非刚体* 的渲染建模
  - 将物质抽象为一团飘忽不定的粒子群
  - 光线在穿过时，是光子在跟粒子发生碰撞的过程
    - 吸收：光子被粒子吸收
    - 放射：粒子本身发光
    - 外射光：光子在冲击后被弹射
    - 内射光：其他方向弹射来的粒子
- NeRF 的假设：
  - 物体是一团自发光的粒子
  - 粒子有 *密度* 和 *颜色*
  - 外射光和内射光抵消
  - 多个粒子被渲染成指定角度的图片
- NeRF 的输入和输出是什么？
  - 模型的输入：
    - 6D 向量 $(x, y, z, d_x, d_y, d_z)$ 
    - 将物体进行稀疏表示的 *单个粒子* 的 *位置* 和 *方向*，即 *位姿*
  - 模型的输出：
    - 4D 向量
    - 该粒子的密度和颜色
  - 【问题】
    - 怎么得到这些粒子？
      - 从图片和相机位姿计算射线
      - 从射线上采样粒子
    - 多少个粒子？这些粒子怎样批量输入？
      - 训练时，一张图片取 1024 个像素，得到 1024 条射线
      - 每条射线上采样 64 个粒子，总共得到 $64 \times 1024$ 个粒子
      - 粒子以 batch 的形式输入模型
    - 这些粒子是怎么渲染成新的图片的？
      - 分别计算图片中每一个像素的颜色
      - 计算该像素对应的光线和粒子
      - 将这些粒子通过体渲染公式累加
      - 得到该像素的最终颜色
- NeRF 的训练流程
  - 输入同一个场景的若干张照片
  - 根据每张照片的每个像素，计算出光线，采样出粒子（粒子有位姿，但没有密度和颜色）
  - 用 NeRF 模型预测这些粒子的密度和颜色
  - 用这些粒子，按照体渲染的方法计算出对应像素的颜色的预测值
  - 像素颜色预测值和 GT 做 MSE
- NeRF 的 推理流程
  - 给定新视角，渲染一张 $400 \times 400$ 的图片
  - 模型输入：$400 \times 400$ 条射线上分别采样 $64$ 个点（自带位置和方向）
  - 模型输出：$(400 \times 400 \times 192, 4)$，这么多个点的颜色和密度
  - 进行体渲染
- NeRF 的总结
  - 隐式渲染（Neural Rendering）的鼻祖论文
  - 核心内容：
    - 体渲染
    - 位置编码
    - adaptive sampling
  - 缺点：
    - 很慢——训练、推理都慢
    - 只能表达静态场景
    - 不支持全局光照
    - 没有泛化能力

- 粒子的采集——生成原理
  - 对于空间上得我某一个发光粒子：
    - 空间坐标 $(x, y, z)$
    - 发射的光线通过 *相机模型* 成为图片上的像素坐标 $(u, v)$
    - 粒子颜色即像素颜色
  - 反之，对于图片上的某一个像素 $(u, v)$ 的颜色
    - 可以看做沿着某一条射线上的无数个发光点的 *和*
    - 利用相机模型，反推射线 $\mathbf{r}(t) = \mathbf{o} + t \mathbf{d}, \ 0 < t < +\infty$
      - $\mathbf{o}$ 为射线原点（即相机位置）
      - $\mathbf{d}$ 为方向（相机位置到 near plane 上这个像素所在位置）
      - $t$ 为距离
      - $\mathbf{V}_{\mathrm{screen}} = (u, v) \to \mathbf{V}_{\mathrm{clip}} = \left( \dfrac{2u}{r - l} - 1, \dfrac{2v}{t - b} - 1, -1, 1 \right) \to  \mathbf{V}_{\mathrm{world}} = \left( \mathbf{P} \, \mathbf{V} \right)^{-1} \, \mathbf{V}_{\mathrm{clip}}$. 
    - 对于整张图片，共有 $(H, W)$ 条射线！

- 粒子的采集——采样原理
  - 采样时 $t$ 的范围不是 $[0, +\infty)$，而是设置了 near = 2，far = 6
  - 在 near 和 far 之间均匀采样 64 个点
  - $t_i = U \left[ t_n + \dfrac{i - 1}{N} (t_f - t_n), t_n + \dfrac{i}{N} (t_f - t_n) \right]$（还加了一点噪声，模拟采样鲁棒性）

- NeRF 的模型结构
  - 概述
    - 8 层全连接层（MLP）
    - 半路再次输入位置坐标
    - 后半路输出密度 $\sigma$
      - 搞定密度之后才输入方向视角的原因：
      - 观测视角不应该影响粒子的密度，只应该影响粒子的颜色 
    - 后半路输入视角坐标
    - 最后输出颜色 RGB
  - 输入是 60 维位置坐标 和 24 维视角坐标
    - 实验发现，只输入 3D 位置和视角时，模型细节丢失（缺乏高频信息）
    - 引入位置编码 $\gamma(p) = \left( \sin(2^0 \pi p), \cos(2^0 \pi p ), \cdots, \sin(2^{L-1} \pi p), \cos(2^{L-1} \pi p) \right)$，共 $2L$ 维
      - $p$ 需要已经归一化到 $[-1, 1]$ 
      - 对空间坐标 $\mathbf{x} = (x, y, z)$，$L = 10$，$\gamma(\mathbf{x})$ 是 60D
      - 对视角坐标 $\mathbf{d} = (d_x, d_y, d_z)$，$L = 4$，$\gamma(\mathbf{d})$ 是 24D
    - 在代码中，加上初始值，$\gamma(\mathbf{x})$ 是 63D，$\gamma(\mathbf{d})$ 是 27D
  - 自监督 Loss
    - GT 为图片某一像素的 RGB
    - 将该像素对应光线上的粒子颜色进行求和
    - 粒子的颜色 和 该像素颜色的预测值 做 MSE $\mathcal{L} = \sum_{r \in R} || \hat{C}(r) - C(r) ||_2^2$
    - R 是每个 batch 的射线（1024 条）
- NeRF 的 粗模型 和 细模型
  - 问题：
    - 无效区域（空白和被遮挡的区域）均匀采样
    - 我们希望有效区域多采样，无效区域少采样或不采样
  - 解决方法：
    - 根据 粗模型预测出的概率密度 进行 再次采样
  - NeRF 模型由两个模型组成，两个 8 层的 MLP 串联到一起
    - 粗模型：输入均匀采样的粒子，输出密度
    - 细模型：根据密度，二次采样
    - 最后输出：采用细模型的输出
    - 粗模型和细模型结构相同
  - 细模型如何进行二次采样？
    - 根据粗模型的结果，进行逆变换采样
      - （如何使得采样结果符合某概率分布，如正态分布？答：均匀分布 + 逆变换采样）
      - 对于每条光线，重新采样 128 个粒子
      - 与之前粗模型采样出的 64 个粒子加在一起
      - 即：每条光线总共采样 192 个粒子
    - 逆变换采样
      - 对于每条射线上的粒子颜色前的权重做 softmax
        - 就是下面 `raw2output` 公式中的 $\alpha_n (1 - \alpha_0) (1 - \alpha_1) \cdots (1 - \alpha_{n - 1})$
      - 此时，新的权重和为 1，可看做 PDF
      - 对 PDF 做前缀和，生成 CDF
      - 计算 CDF 的反函数
      - 用均匀分布生成一个随机数，`invert_cdf(uniform_real_distribution(0, 1)) = r`
      - 得到的 r 就是符合 PDF 分布的随机数

- 体渲染：对光线上的粒子颜色进行求和
  - 连续版本：
    - $\displaystyle C = \int_0^{+\infty} T(s) \sigma(s) C(s) \mathrm{d}s = \sum_i T_i \alpha_i C_i$
    - $T(s)$：在 s 点之前，光线没有被阻挡的概率，【由两个已知求出来】
      - $T(s) = \exp(-\int_0^s \sigma(t) \mathrm{d}t)$，这是按概率论列微分方程解出来的解析解
    - $\sigma(s)$：在 s 点处，光线碰撞粒子（光线被粒子阻碍）的概率，【已知】
    - $C(s)$：在 s 点处，粒子的颜色，【已知】
  - 离散化：
    - 将光线 $[0, s]$ 划分为 N 个等间距区间 $[T_n, T_{n+1}]$，$n = 0, 1, \cdots, N$
      - 间隔长度为 $\delta_n$
      - 假设区间内的密度 $\sigma_n$ 和颜色 $C_n$ 固定
    - $\displaystyle C(r) = \sum_{n=0}^N C_n e^{- \sum_{i=0}^{n-1} \sigma_i \delta_i} (1 - e^{- \sigma_n \delta_n}) = \sum_n C_n T_n \alpha_n$，其中：
      - $C_n$ 叫颜色，是之前通过球谐函数算出来的 RGB 值
      - $T_n = e^{- \sum_{i=0}^{n-1} \sigma_i \delta_i}$ 叫没有被阻挡的概率
        - $T_n = 0$ 的话直接不用算了，因为被挡住了
      - $\alpha_n = 1 - e^{- \sigma_n \delta_n}$ 叫不透明度
  - 代码中对公式做了进一步简化（见函数 `raw2output`）：
    - $\displaystyle \hat{C} = \sum_{n=0}^N C_n e^{- \sum_{j=1}^{i-1} \sigma_j \delta_j} (1 - e^{- \sigma_i \delta_i}) = \sum_{n=0}^N C_n T_n \alpha_n$
    - 其中 $T_n = 
    e^{- \sum_{j=1}^{i-1} \sigma_j \delta_j} = 
    e^{ - \left(\sigma_0 \delta_0 + \sigma_1 \delta_1 + \sigma_2 \delta_2 + \cdots + \sigma_{n-1} \delta_{n-1} \right) } = 
    e^{- \sigma_0 \delta_0} \cdot e^{- \sigma_1 \delta_1} \cdot e^{- \sigma_2 \delta_2} \cdots e^{- \sigma_{n-1} \delta_{n-1}}
    $
    - 在上式中代入 $\alpha_n = 1 - e^{- \sigma_n \delta_n}$，得：$T_n = (1 - \alpha_0) (1 - \alpha_1) (1 - \alpha_2) \cdots (1 - \alpha_{n - 1})$
    - 则 $
    \displaystyle 
    \begin{aligned} 
        \hat{C} & = \sum_{n=0}^N C_n \alpha_n (1 - \alpha_0) (1 - \alpha_1) \cdots (1 - \alpha_{n - 1}) \\ 
                & = C_0 \alpha_0 + C_1 \alpha_1 (1 - \alpha_0) + C_2 \alpha_2 (1 - \alpha_0) (1 - \alpha_1) + \cdots + C_n \alpha_n (1 - \alpha_0) (1 - \alpha_1) \cdots (1 - \alpha_{n - 1})
    \end{aligned}
    $